In [1]:
import functools

# 全局变量，控制输出信息的详细程度
INFO_DENSITY = 2

def controlled_print_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # 替换内置的print函数
        original_print = print
        
        def custom_print(level, *print_args, **print_kwargs):
            if level <= INFO_DENSITY:
                original_print(*print_args, **print_kwargs)
        
        # 将全局 print 替换为 custom_print
        globals()['print'] = custom_print
        
        try:
            result = func(*args, **kwargs)
        finally:
            # 恢复原始的print函数
            globals()['print'] = original_print
        
        return result
    
    return wrapper

# 示例使用装饰器
@controlled_print_decorator
def example_function():
    print(1, "This is a level 1 message.")
    print(2, "This is a level 2 message.")
    print(3, "This is a level 3 message.")
    print(4, "This is a level 4 message.")

# 修改 INFO_DENSITY 测试不同的输出
INFO_DENSITY = 2
example_function()

INFO_DENSITY = 3
example_function()

INFO_DENSITY = 4
example_function()

This is a level 1 message.
This is a level 2 message.
This is a level 1 message.
This is a level 2 message.
This is a level 3 message.
This is a level 1 message.
This is a level 2 message.
This is a level 3 message.
This is a level 4 message.


管理信息输出的详细程度是软件开发中的一个常见需求，尤其是在调试和生产环境中。除了刚刚介绍的基于装饰器的方法，还有其他几种常用的方案可以帮助你更好地管理输出信息的详细程度。这些方案包括使用 Python 的日志库（logging）、基于环境变量的控制、以及自定义的输出管理类。下面是这些方案的详细介绍：

1. 使用 Python 的 logging 库
logging 是 Python 标准库中非常强大的模块，能够细粒度地控制日志的输出，包括不同的日志级别（如 DEBUG、INFO、WARNING、ERROR、CRITICAL）。这是一个比较推荐的做法，因为它提供了灵活性和可扩展性。

In [2]:
import logging

# 配置 logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 设置全局的详细程度控制
INFO_DENSITY = 2  # 0 - NOTSET, 1 - DEBUG, 2 - INFO, 3 - WARNING, 4 - ERROR, 5 - CRITICAL

def set_logging_level(density):
    if density == 0:
        logger.setLevel(logging.NOTSET)
    elif density == 1:
        logger.setLevel(logging.DEBUG)
    elif density == 2:
        logger.setLevel(logging.INFO)
    elif density == 3:
        logger.setLevel(logging.WARNING)
    elif density == 4:
        logger.setLevel(logging.ERROR)
    elif density == 5:
        logger.setLevel(logging.CRITICAL)
    else:
        logger.setLevel(logging.INFO)

set_logging_level(INFO_DENSITY)

def example_function():
    logger.debug("This is a debug message.")
    logger.info("This is an info message.")
    logger.warning("This is a warning message.")
    logger.error("This is an error message.")
    logger.critical("This is a critical message.")

example_function()


2024-08-10 00:16:10,657 - INFO - This is an info message.
2024-08-10 00:16:10,658 - WARNING - This is a warning message.
2024-08-10 00:16:10,658 - ERROR - This is an error message.
2024-08-10 00:16:10,658 - CRITICAL - This is a critical message.


优点
灵活性：logging 提供了多种日志级别和格式化选项，适合不同的需求。
可扩展性：支持将日志输出到控制台、文件、甚至远程日志服务器。
集成性：可以与现有的许多 Python 库和框架无缝集成。
适用场景
需要在项目中管理复杂的日志输出，且要求支持不同的输出目标（如控制台、文件）。
需要不同模块或类中统一管理输出信息。
2. 基于环境变量的控制
你可以使用环境变量来动态控制输出信息的详细程度。这样可以在不同的运行环境中方便地调整输出的级别，而无需修改代码。

In [3]:
import os

INFO_DENSITY = int(os.getenv("INFO_DENSITY", 2))  # 从环境变量获取详细程度，默认为2

def controlled_print(level, message):
    if level <= INFO_DENSITY:
        print(message)

def example_function():
    controlled_print(1, "This is a level 1 message.")
    controlled_print(2, "This is a level 2 message.")
    controlled_print(3, "This is a level 3 message.")
    controlled_print(4, "This is a level 4 message.")

example_function()


This is a level 1 message.
This is a level 2 message.


优点
动态调整：可以在运行时通过设置环境变量来控制信息的输出详细程度。
无侵入性：无需更改代码逻辑，只需通过环境变量控制输出。
适用场景
需要在开发、测试、生产等不同环境中动态调整输出的详细程度。
项目对环境变量有良好的支持。
3. 自定义输出管理类
你可以创建一个自定义的输出管理类，用于集中管理所有的输出逻辑。这个类可以使用单例模式（Singleton），确保在整个应用中只有一个实例。

In [4]:
class OutputManager:
    _instance = None

    def __new__(cls, info_density=2):
        if cls._instance is None:
            cls._instance = super(OutputManager, cls).__new__(cls)
            cls._instance.info_density = info_density
        return cls._instance

    def set_info_density(self, density):
        self.info_density = density

    def print(self, level, message):
        if level <= self.info_density:
            print(message)

# 使用示例
output_manager = OutputManager(info_density=2)

def example_function():
    output_manager.print(1, "This is a level 1 message.")
    output_manager.print(2, "This is a level 2 message.")
    output_manager.print(3, "This is a level 3 message.")
    output_manager.print(4, "This is a level 4 message.")

example_function()

# 动态修改详细程度
output_manager.set_info_density(3)
example_function()


This is a level 1 message.
This is a level 2 message.
This is a level 1 message.
This is a level 2 message.
This is a level 3 message.


优点
集中管理：所有的输出控制都集中在一个类中，易于维护。
动态调整：可以在运行过程中动态调整输出的详细程度。
适用场景
需要在项目中集中管理输出逻辑。
需要根据运行时条件动态调整输出行为。
总结
logging 库：是最灵活和功能最强大的解决方案，适合管理复杂的日志需求。
环境变量：提供了一种简单、动态的方法来控制输出的详细程度，适合需要在不同环境中运行的项目。
自定义输出管理类：适合在项目中集中管理输出逻辑，易于扩展和维护。
如果你需要一个强大且灵活的解决方案，推荐使用 Python 的 logging 库。如果你想要一个轻量级且可以动态调整的方式，基于环境变量的控制或自定义输出管理类都是不错的选择。